In [ ]:
from turtle import forward
import numpy as np
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.utils.data as Data
import data_loader as dl
import train as tt
import sys
import os
import random as rd

In [ ]:
def copydir(n):
    os.system(f"cp data0 data{n} -r")
    os.system(f"cp generate_data0 generate_data{n} -r")
    item=os.listdir(f"data{n}")
    os.mkdir(f"data{n}/train")
    os.mkdir(f"data{n}/test")
    for i in item:
        if ".npy" in i:
            data = np.load(f"data{n}/{i}")
            state = np.random.uniform(0,1,size=(len(data)))
            train = data[state >= 0.2]
            test = data [state < 0.2]
            np.save(f"data{n}/train/{i}",train)
            np.save(f"data{n}/test/{i}",test)
    
             

In [ ]:
net=tt.sgrna_net().cuda()
#net=torch.load("generate_data/model/500.pt")
lossf=nn.SmoothL1Loss(reduction="mean")
print(net)

In [ ]:
for datat in range(1,20):
    copydir(datat)
    torch.manual_seed(1024)
    net=tt.sgrna_net().cuda()
    train=dl.data_loader(dic=f"./data{datat}/train")
    #train,va = Data.random_split(data,[int(len(data)*0.8),len(data)-int(len(data)*0.8)])
    test=dl.data_loader(dic=f"./data{datat}/test")
    trloader=Data.DataLoader(dataset=train,batch_size=5000,shuffle=True)
    #valoader=Data.DataLoader(dataset=va ,batch_size=5000,shuffle=True)
    testloader = Data.DataLoader(dataset=test,batch_size=1000,shuffle=True)
    opt=torch.optim.Adam(filter(lambda p : p.requires_grad, net.parameters()),lr=0.001)
    she=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt,T_0=5)   
    steps=40000
    wt1=open(f"./generate_data{datat}/traj/trloss","w+")
    wt2=open(f"./generate_data{datat}/traj/valoss","w+")
    #net=torch.load("generate_data/model/500.pt")
    lossf=nn.SmoothL1Loss(reduction="mean")
    print(net)
    for _ in range(steps):
        net.train()
        for seq,typ,eff in trloader:
            seq=seq.cuda()
            typ=typ.cuda()
            eff=eff.cuda()
            eff+=(torch.rand(size=eff.size(),device="cuda")-0.5)*0.1
            eff =torch.abs(eff)
            pre=net(seq,typ)
            loss=lossf(pre,eff)
            opt.zero_grad()
            loss.backward()
            opt.step()
            she.step()
        print(loss.detach().cpu().numpy(),file=wt1,flush=True)

        net.eval()
        for seq,typ,eff in testloader:
            seq=seq.cuda()
            typ=typ.cuda()
            eff=eff.cuda()
            pre=net(seq,typ)
            loss=lossf(pre,eff)
        print(loss.detach().cpu().numpy(),file=wt2,flush=True)

        if _%100 ==0:
            presave=[]
            effsave=[]
            for seq,typ,eff in trloader:
                seq=seq.cuda()
                typ=typ.cuda()
                eff=eff.cuda()
                pre=net(seq,typ)
                pre=pre.detach().cpu().numpy()
                eff=eff.detach().cpu().numpy()
                if len(presave)==0:
                    presave=np.copy(pre)
                    effsave=np.copy(eff)
                else:
                    presave=np.concatenate((presave,pre))
                    effsave=np.concatenate((effsave,eff))
                
            np.save(f"./generate_data{datat}/out/effv"+"_"+str(_),effsave)
            np.save(f"./generate_data{datat}/out/prev"+"_"+str(_),presave)

            presave=[]
            effsave=[]
            for seq,typ,eff in testloader:
                seq=seq.cuda()
                typ=typ.cuda()
                eff=eff.cuda()
                pre=net(seq,typ)
                pre=pre.detach().cpu().numpy()
                eff=eff.detach().cpu().numpy()
                if len(presave)==0:
                    presave=np.copy(pre)
                    effsave=np.copy(eff)
                else:
                    presave=np.concatenate((presave,pre))
                    effsave=np.concatenate((effsave,eff))
                
            np.save(f"./generate_data{datat}/out/eff"+"_"+str(_),effsave)
            np.save(f"./generate_data{datat}/out/pre"+"_"+str(_),presave)

            torch.save(net,f"./generate_data{datat}/model/"+str(_)+".pt")